In [146]:
import streamlit as st
import pandas as pd
import altair as alt
from sqlalchemy import create_engine
from scripts.config import DB_CONNECTION_STRING

In [147]:
engine = create_engine(DB_CONNECTION_STRING)
city = "Tokyo"
sql = "SELECT * FROM weather_daily WHERE city = %s"
df_daily = pd.read_sql(sql=sql, con=engine, params=(city,))
df_daily.head()

,date,city,temperature_max,temperature_min,precipitation_sum,weather_code
0,2020-01-01 00:00:00+00:00,Tokyo,8.25,0.00,0.0,3.0
1,2020-01-02 00:00:00+00:00,Tokyo,11.50,-1.20,0.0,3.0
2,2020-01-03 00:00:00+00:00,Tokyo,10.30,2.70,0.0,1.0
3,2020-01-04 00:00:00+00:00,Tokyo,10.90,1.15,5.6,73.0
4,2020-01-05 00:00:00+00:00,Tokyo,9.35,2.15,0.0,2.0


In [148]:
df_daily = df_daily.rename(columns={
    'temperature_max': 'temp_max',
    'temperature_min': 'temp_min',
})

In [149]:
df_daily['weather_code'].unique()

array([ 3.,  1., 73.,  2., 53., 63., 51., 55., 71., 65.,  0., 61., 75.])

In [150]:
weather_map = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    51: "Light drizzle",
    53: "Moderate drizzle",
    55: "Dense drizzle",
    61: "Slight rain",
    63: "Moderate rain",
    65: "Heavy rain",
    71: "Slight snow fall",
    73: "Moderate snow fall",
    75: "Heavy snow fall"
}

# Weather short name mapping
weather_short_map = {
    0: "sun",
    1: "sun",
    2: "sun",
    3: "cloud",
    51: "drizzle",
    53: "drizzle",
    55: "drizzle",
    61: "rain",
    63: "rain",
    65: "rain",
    71: "snow",
    73: "snow",
    75: "snow"
}

df_daily['weather_desc'] = df_daily['weather_code'].map(weather_map)
df_daily['weather_short'] = df_daily['weather_code'].map(weather_short_map)
df_daily.head()


,date,city,temp_max,temp_min,precipitation_sum,weather_code,weather_desc,weather_short
0,2020-01-01 00:00:00+00:00,Tokyo,8.25,0.00,0.0,3.0,Overcast,cloud
1,2020-01-02 00:00:00+00:00,Tokyo,11.50,-1.20,0.0,3.0,Overcast,cloud
2,2020-01-03 00:00:00+00:00,Tokyo,10.30,2.70,0.0,1.0,Mainly clear,sun
3,2020-01-04 00:00:00+00:00,Tokyo,10.90,1.15,5.6,73.0,Moderate snow fall,snow
4,2020-01-05 00:00:00+00:00,Tokyo,9.35,2.15,0.0,2.0,Partly cloudy,sun


In [151]:
df_daily['date'] = pd.to_datetime(df_daily['date'])
df_daily['year'] = df_daily['date'].dt.year
#df_daily = df_daily[df_daily['year'] == 2022]
df_daily.head()

,date,city,temp_max,temp_min,precipitation_sum,weather_code,weather_desc,weather_short,year
0,2020-01-01 00:00:00+00:00,Tokyo,8.25,0.00,0.0,3.0,Overcast,cloud,2020
1,2020-01-02 00:00:00+00:00,Tokyo,11.50,-1.20,0.0,3.0,Overcast,cloud,2020
2,2020-01-03 00:00:00+00:00,Tokyo,10.30,2.70,0.0,1.0,Mainly clear,sun,2020
3,2020-01-04 00:00:00+00:00,Tokyo,10.90,1.15,5.6,73.0,Moderate snow fall,snow,2020
4,2020-01-05 00:00:00+00:00,Tokyo,9.35,2.15,0.0,2.0,Partly cloudy,sun,2020


In [152]:
import plotly.express as px

fig = px.line(
    df_daily,
    x='date',
    y='temp_min',
    labels={
        'date': 'date',
        'temp_min': 'Temperature_min (°C)',
        'year': 'year'
    },
    title="Weather Tokyo 2022"
)
fig.update_layout(
    hovermode='x unified',
    xaxis=dict(
        tickformat="%b %d"
    )
)
fig.update_traces(line_color='red')
fig


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'date=%{x}<br>Temperature_min (°C)=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': 'red', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array(['2020-01-01T00:00:00.000000000', '2020-01-02T00:00:00.000000000',
                          '2020-01-03T00:00:00.000000000', ..., '2026-01-26T00:00:00.000000000',
                          '2026-01-27T00:00:00.000000000', '2026-01-28T00:00:00.000000000'],
                         shape=(2220,), dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAAAAzMzMzMzPzv5qZmZmZmQ' ... 'mZmZmZ9T+amZmZmZn5PwAAAAAAAPg/'),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'hovermode': 'x unified',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Weather Tokyo 2022'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'tickformat': '%b %d', 'title': {'text': 'date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Temperature_min (°C)'}}}
})

In [153]:
weather_count = df_daily['weather_short'].value_counts().reset_index()
weather_count.columns = ['weather', 'count']
weather_count


,weather,count
0,cloud,761
1,drizzle,687
2,rain,476
3,sun,247
4,snow,49


In [154]:
fig = px.pie(
    weather_count,
    names='weather',
    values='count',
    title='Weather Distribution in Tokyo 2022',
    color_discrete_sequence=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']
)
fig.update_traces(textinfo='label', textposition='auto')
fig


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': 'weather=%{label}<br>count=%{value}<extra></extra>',
              'labels': array(['cloud', 'drizzle', 'rain', 'sun', 'snow'], dtype=object),
              'legendgroup': '',
              'name': '',
              'showlegend': True,
              'textinfo': 'label',
              'textposition': 'auto',
              'type': 'pie',
              'values': {'bdata': '+QKvAtwB9wAxAA==', 'dtype': 'i2'}}],
    'layout': {'legend': {'tracegroupgap': 0},
               'piecolorway': [#FF6B6B, #4ECDC4, #45B7D1, #FFA07A, #98D8C8],
               'template': '...',
               'title': {'text': 'Weather Distribution in Tokyo 2022'}}
})

In [155]:
# # Chuẩn bị dữ liệu
# df_daily['date'] = pd.to_datetime(df_daily['date'])
# df_daily['year'] = df_daily['date'].dt.year
# df_daily['month_day'] = df_daily['date'].dt.strftime('%b %d')
# df_daily['day_of_year'] = df_daily['date'].dt.dayofyear

# # Lọc dữ liệu theo năm mà bạn muốn so sánh
# years_to_compare = [2020]
# df_filtered = df_daily[df_daily['year'].isin(years_to_compare)].copy()

# # Sắp xếp theo ngày trong năm
# df_filtered = df_filtered.sort_values('day_of_year')

# df_filtered.head(10)

In [156]:
# # Tạo biểu đồ so sánh năm
# import plotly.graph_objects as go
# import plotly.express as px

# # Tính toán ngày tháng dưới dạng string cho trục X (Jan 01, Feb 01, ...)
# df_filtered['date_str'] = df_filtered['date'].dt.strftime('%b %d')

# # Tạo biểu đồ line interactive
# fig = px.line(
#     df_filtered, 
#     x='date_str', 
#     y='temperature_max',
#     color='year',
#     title='So sánh nhiệt độ theo năm',
#     labels={
#         'temperature_max': 'Nhiệt độ (°C)',
#         'date_str': 'Ngày',
#         'year': 'Năm'
#     },
#     hover_data={'temperature_max': ':.1f', 'date_str': True, 'year': True},
#     color_discrete_sequence=['#1f77b4', '#87ceeb', '#ff7f0e', '#2ca02c']
# )

# # Cập nhật layout
# fig.update_layout(
#     hovermode='x unified',
#     height=600,
#     template='plotly_white',
#     xaxis_title='Ngày',
#     yaxis_title='Nhiệt độ (°C)',
#     font=dict(size=12)
# )

# # Hiển thị biểu đồ
# fig


In [157]:
# # Widget để chọn năm (nếu muốn dùng với Streamlit)
# # Tất cả năm có sẵn
# all_years = sorted(df_daily['year'].unique())
# print(f"Các năm có sẵn: {all_years}")

# # Chọn năm mặc định
# selected_years = [2025, 2026]

# # Lọc dữ liệu cho biểu đồ tương tác
# df_selected = df_daily[df_daily['year'].isin(selected_years)].copy()
# df_selected['date_str'] = df_selected['date'].dt.strftime('%b %d')
# df_selected = df_selected.sort_values('date')

# # Tạo biểu đồ với Altair (tương thích với Streamlit)
# chart = alt.Chart(df_selected).mark_line().encode(
#     x=alt.X('date_str:N', title='Ngày'),
#     y=alt.Y('temperature_max:Q', title='Nhiệt độ Max (°C)'),
#     color=alt.Color('year:N', title='Năm'),
#     tooltip=['date_str:N', 'temperature_max:Q', 'year:N']
# ).properties(
#     width=1000,
#     height=600,
#     title='So sánh nhiệt độ theo năm'
# ).interactive()

# chart
